<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao24_FastText_nativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem 2: Classificação com o FastText Nativo

Esta é a segunda forma de usar o FastText: como um classificador *completo* e autônomo. Esta abordagem **não usa Keras nem LSTM**.

O FastText (biblioteca `fasttext`) é otimizado para ser extremamente rápido e eficiente, muitas vezes superando LSTMs complexas em tarefas de classificação simples, com um tempo de treino ordens de magnitude menor.

**Fluxo:**
1. Instalar `fasttext`.
2. Carregar os dados (vamos usar o mesmo dataset).
3. Formatar os dados no formato de arquivo esperado pelo FastText.
4. Treinar o classificador (geralmente uma única linha de comando).
5. Avaliar o classificador.

In [14]:
# Célula 1: Instalação e Importações

# Instala a biblioteca oficial do Facebook
!pip install fasttext

import fasttext
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import re
import os

## Passo 1: Carregar Dados

Carregamos os mesmos dados de treino e teste.

In [15]:
# Célula 2: Carregar os dados
categorias = ['comp.graphics', 'sci.crypt']
dados = fetch_20newsgroups(subset='all', categories=categorias, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
X_train_text, X_test_text, y_train, y_test = train_test_split(dados.data, dados.target, test_size=0.2, random_state=42)

print(f"Amostras de treino: {len(X_train_text)}")
print(f"Amostras de teste: {len(X_test_text)}")

Amostras de treino: 1571
Amostras de teste: 393


## Passo 2: Formatar Dados para o FastText

O FastText espera um arquivo de texto simples onde cada linha é um documento, formatado como:

`__label__<ROTULO> <texto do documento pré-processado>`

Vamos criar `train.txt` e `test.txt` nesse formato.

In [16]:
# Célula 3: Pré-processamento e formatação

def preprocess_text_fasttext(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text) # Remove não-alfanuméricos
    text = re.sub(r'\s+', ' ', text) # Remove espaços extras
    return text.strip()

def criar_arquivo_fasttext(filename, textos, rotulos):
    with open(filename, 'w', encoding='utf-8') as f:
        for texto, rotulo in zip(textos, rotulos):
            texto_limpo = preprocess_text_fasttext(texto)
            if texto_limpo:
                # O formato é __label__<numero> texto...
                f.write(f"__label__{rotulo} {texto_limpo}\n")

# Criar os arquivos de treino e teste
criar_arquivo_fasttext('fasttext_train.txt', X_train_text, y_train)
criar_arquivo_fasttext('fasttext_test.txt', X_test_text, y_test)

print("Arquivos fasttext_train.txt e fasttext_test.txt criados.")

# Vamos ver as primeiras linhas do arquivo de treino
!head -n 3 fasttext_train.txt

Arquivos fasttext_train.txt e fasttext_test.txt criados.
__label__0 hello there a week ago a guy asked what a sco file was well i researched a bit and foundout that it is just another rix file sci files are 320x200 files and sco files are 1024x768 files alle the other formats 800x600 640x480 are also called something like sc character
__label__0 it should have been made fairly clear that the most crimson would ever get was a 150 75 old style mhz cpu upgrade certainly this was mentioned on comp sys sgi on more than one occasion as being likely if our sales folks were saying otherwise they were either confused or less than honest ethical or somebody further up the chain inside sgi was misleading them there were 4 versions 20 25 30 35 although admittedly the 30 came out at the same time as the 35 over a period of 2 1 2 years the chassis simply couldn t be pushed any further i d say 4 years was a pretty good lifespan myself for a system design in this day and age getting the 35 to work cau

## Passo 3: Treinar o Classificador Nativo

Este é o passo mágico. O treino é feito com uma única função `train_supervised`.

In [17]:
# Célula 4: Treinamento

print("Treinando o classificador nativo FastText...")

# Treina o modelo
# 'wordNgrams=2' informa ao FastText para usar bigramas, o que ajuda muito na classificação
# 'epoch=25' e 'lr=1.0' são hiperparâmetros comuns e muito bons
modelo_nativo = fasttext.train_supervised(
    input='fasttext_train.txt',
    epoch=25,
    lr=1.0,
    wordNgrams=2,
    loss='softmax'
)

print("Treinamento concluído.")

# Salvar o modelo (opcional)
modelo_nativo.save_model("modelo_fasttext_classificador.bin")

Treinando o classificador nativo FastText...
Treinamento concluído.


## Passo 4: Avaliar e Testar o Modelo

In [18]:
# Célula 5: Avaliação

# A função .test() retorna (N_amostras, precisão, recall)
# No fasttext, 'precisão' neste contexto é a acurácia (P@1)
resultado = modelo_nativo.test('fasttext_test.txt')

n_amostras = resultado[0]
acuracia = resultado[1] # P@1 é a acurácia
recall = resultado[2]   # R@1 é o recall

print(f"Amostras de teste: {n_amostras}")
print(f"Acurácia (FastText Nativo): {acuracia:.4f}")
print(f"Recall (FastText Nativo): {recall:.4f}")

Amostras de teste: 380
Acurácia (FastText Nativo): 0.9000
Recall (FastText Nativo): 0.9000


## Conclusão

Você provavelmente notará que a Abordagem 2 (Nativa) não só treinou em **segundos**, como também atingiu uma acurácia extremamente alta, muitas vezes comparável ou superior à da LSTM, com muito menos complexidade.

Para problemas de classificação de texto, o classificador nativo do FastText é uma *baseline* (ponto de partida) incrivelmente forte.